# Algoritmos de conjunto

https://scikit-learn.org/stable/modules/classes.html#module-sklearn.ensemble

* Bagging
    * BaggingClassifier
    * BaggingRegressor
    * RandomForestClassifier
    * RandomForestRegressor
    * ExtraTreesClassifier
    * ExtraTreesRegressor

* Voting
	* VotingClassifier
    * VotingRegressor

* Boosting
    * AdaBoostClassifier
    * AdaBoostRegressor
    * GradientBoostingClassifier
    * GradientBoostingRegressor

* Stacking
	* StackingClassifier
	* StackingRegressor

In [17]:
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error, mean_absolute_percentage_error

df = sns.load_dataset('mpg').dropna()  # Eliminamos filas con valores nulos

features = ['weight', 'cylinders', 'displacement', 'horsepower', 'acceleration', 'model_year']
X = df[features]
y = df['mpg']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 1. Bagging (Bootstrap Aggregating)

- **Idea principal**: Entrenar varios modelos (generalmente iguales) en muestras *bootstrap* (subconjuntos del dataset con reemplazo) y luego combinar sus predicciones (votación mayoritaria o promedio).  Construye múltiples modelos a partir de diferentes submuestras del conjunto de datos de entrenamiento.
- **Objetivo**: Reducir la varianza y mejorar la estabilidad.  
- **Ejemplos en Scikit-Learn**:
  - `BaggingClassifier`, `BaggingRegressor`: Métodos genéricos de bagging para cualquier estimador base.
  - `RandomForestClassifier`, `RandomForestRegressor`: Casos populares de bagging con árboles y muestreo de características.
  - `ExtraTreesClassifier`, `ExtraTreesRegressor`: Similar a RandomForest pero con mayor aleatoriedad al dividir nodos.

In [18]:
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, ExtraTreesRegressor

models_bagging = {
    "BaggingRegressor": BaggingRegressor(random_state=42),
    "RandomForestRegressor": RandomForestRegressor(random_state=42),
    "ExtraTreesRegressor": ExtraTreesRegressor(random_state=42)
}

for name, model in models_bagging.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"{name}: R2 = {r2_score(y_test, y_pred):.3f}, MAE = {mean_absolute_error(y_test, y_pred):.3f}, RMSE = {root_mean_squared_error(y_test, y_pred):.3f}, MAPE = {mean_absolute_percentage_error(y_test, y_pred):.3f}")

BaggingRegressor: R2 = 0.842, MAE = 2.049, RMSE = 2.838, MAPE = 0.091
RandomForestRegressor: R2 = 0.885, MAE = 1.761, RMSE = 2.423, MAPE = 0.080
ExtraTreesRegressor: R2 = 0.896, MAE = 1.693, RMSE = 2.305, MAPE = 0.076


## 2. Voting

- **Idea principal**: es un complemento de bagging. Combina diferentes estimadores (pueden ser modelos distintos) entrenados en el **mismo** conjunto de datos.  
- **Cómo se combinan**:  
  - Clasificación: votación por mayoría (hard) o promedio de probabilidades (soft).  
  - Regresión: se hace un promedio (o combinación) de las salidas.  
- **Ejemplos en Scikit-Learn**:
  - `VotingClassifier`  
  - `VotingRegressor`

In [19]:
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor

model1 = LinearRegression()
model2 = KNeighborsRegressor()
model3 = RandomForestRegressor(random_state=42)

voting_reg = VotingRegressor([
    ("lr", model1),
    ("knn", model2),
    ("rf", model3)
])

voting_reg.fit(X_train, y_train)
y_pred = voting_reg.predict(X_test)
print(f"VotingRegressor: R2 = {r2_score(y_test, y_pred):.3f}, MAE = {mean_absolute_error(y_test, y_pred):.3f}, RMSE = {root_mean_squared_error(y_test, y_pred):.3f}, MAPE = {mean_absolute_percentage_error(y_test, y_pred):.3f}")

VotingRegressor: R2 = 0.834, MAE = 2.119, RMSE = 2.909, MAPE = 0.096


## 3. Boosting

- **Idea principal**: Entrenar en secuencia múltiples “weak learners”; cada nuevo modelo se centra en corregir los errores de los modelos anteriores.  
- **Cómo funciona**: Ajustar iterativamente modelos base (por ejemplo, árboles pequeños) dando más peso a los errores o los residuos en cada paso.  
- **Ejemplos en Scikit-Learn**:
  - `AdaBoostClassifier`, `AdaBoostRegressor`: Ajusta pesos de los ejemplos para corregir errores.  
  - `GradientBoostingClassifier`, `GradientBoostingRegressor`: Ajuste secuencial basado en el gradiente de la función de pérdida.

In [20]:
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor

models_boosting = {
    "AdaBoostRegressor": AdaBoostRegressor(random_state=42),
    "GradientBoostingRegressor": GradientBoostingRegressor(random_state=42)
}

for name, model in models_boosting.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"{name}: R2 = {r2_score(y_test, y_pred):.3f}, MAE = {mean_absolute_error(y_test, y_pred):.3f}, RMSE = {root_mean_squared_error(y_test, y_pred):.3f}, MAPE = {mean_absolute_percentage_error(y_test, y_pred):.3f}")

AdaBoostRegressor: R2 = 0.818, MAE = 2.240, RMSE = 3.048, MAPE = 0.103
GradientBoostingRegressor: R2 = 0.872, MAE = 1.822, RMSE = 2.551, MAPE = 0.082


## 4. Stacking (Stacked Generalization)

- **Idea principal**: Entrenar varios modelos base y luego usar **sus predicciones** como entradas para un modelo “meta” que aprende la mejor forma de combinarlas.  
- **Cómo funciona**: 
  1. Entrenar los modelos base.  
  2. Usar sus predicciones para alimentar un segundo modelo.  
  3. El metamodelo combina los outputs de los anteriores.  
- **Ejemplos en Scikit-Learn**:
  - `StackingClassifier`  
  - `StackingRegressor`

In [21]:
from sklearn.ensemble import StackingRegressor
from sklearn.tree import DecisionTreeRegressor

base_estimators = [
    ("lr", LinearRegression()),
    ("dt", DecisionTreeRegressor(random_state=42))
]

model = StackingRegressor(
    estimators=base_estimators,
    final_estimator=RandomForestRegressor(random_state=42)
)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f"StackingRegressor: R2 = {r2_score(y_test, y_pred):.3f}, MAE = {mean_absolute_error(y_test, y_pred):.3f}, RMSE = {root_mean_squared_error(y_test, y_pred):.3f}, MAPE = {mean_absolute_percentage_error(y_test, y_pred):.3f}")

StackingRegressor: R2 = 0.853, MAE = 1.952, RMSE = 2.737, MAPE = 0.086
